# 使用 pymongo 操作 MongoDB

[PyMongo 官方文档 - 翻译](https://www.cnblogs.com/zhouxuchen/p/5544227.html)

## 增

In [3]:
from pymongo import MongoClient

class AddMongo(object):
    
    def __init__(self):
        '''建立连接'''
        self.client = MongoClient('mongodb://localhost:27017')
        self.db = self.client['demo']
    
    def add_one(self):
        '''新增一条数据'''
        post = { 'name':"bob", 'age':16, 'sex':"male", 'grade':95 }
        return self.db.demo.test.insert_one(post)
        
        
    def add_many(self):
        '''新增多条数据'''
        return self.db.demo.test.insert_many(
            [
                { 'name':"bob", 'age':16, 'sex':"male", 'grade':95},
                { 'name':"ahn", 'age':18, 'sex':"female", 'grade':45},
                { 'name':"xi", 'age':15, 'sex':"male", 'grade':75}
            ]
        )

    
def main():
    obj = AddMongo()
    #rest = obj.add_one()
    #print(rest.inserted_id)
    rest = obj.add_many()
    print(rest.inserted_ids)

    
if __name__ == '__main__':
    main()

[ObjectId('5e475ba304ca2e36d435f7dc'), ObjectId('5e475ba304ca2e36d435f7dd'), ObjectId('5e475ba304ca2e36d435f7de')]


## 查

In [6]:
from pymongo import MongoClient
from bson.objectid import ObjectId

class SearchMongo(object):
    
    def __init__(self):
        '''建立连接'''
        self.client = MongoClient('mongodb://localhost:27017')
        self.db = self.client['demo']
    
    def get_one(self):
        '''查询一条效据'''
        return self.db.demo.test.find_one()

    def get_more(self):
        '''查询多条数据'''
        return self.db.demo.test.find({'sex': 'male'})
    
    def get_from_oid(self, oid):
        '''根据记录的ID来获取数据'''
        #from bson.objectid import ObjectId
        return self.db.demo.test.find_one({'_id': ObjectId(oid)})

    
def main():
    obj = SearchMongo()
    #rest = obj.get_one()
    #print(rest)
    rest = obj.get_more()
    for item in rest:
        print(item)
    #rest = obj.get_from_oid('5e44061d04ca2e36ac2580c0')
    #print(rest)

    
if __name__ == '__main__':
    main()

{'_id': ObjectId('5e44061d04ca2e36ac2580c0'), 'name': 'bob', 'age': 16, 'sex': 'male', 'grade': 95}
{'_id': ObjectId('5e44061d04ca2e36ac2580c2'), 'name': 'xi', 'age': 15, 'sex': 'male', 'grade': 75}


## 改

In [11]:
from pymongo import MongoClient

class ModifyMongo(object):
    
    def __init__(self):
        '''建立连接'''
        self.client = MongoClient('mongodb://localhost:27017')
        self.db = self.client['demo']
    
    def update_one(self):
        '''修改一条数据'''
        rest = self.db.demo.test.update_one({'name': "bob"}, {'$set': {'age':20}})
        return rest
    
    def update_many(self):
        '''修改多条数据'''
        rest = self.db.demo.test.update_many({'sex': 'male'}, {'$inc': {'grade':1}}) # 所有男性成绩增加1
        return rest

    
def main():
    obj = ModifyMongo()
    rest = obj.update_one()
    #rest = obj.update_many()
    print(rest.matched_count)
    print(rest.modified_count)
    
    
if __name__ == '__main__':
    main()

1
1


## 删

In [13]:
from pymongo import MongoClient

class DeleteMongo(object):
    
    def __init__(self):
        '''建立连接'''
        self.client = MongoClient('mongodb://localhost:27017')
        self.db = self.client['demo']
    
    def delete_one(self):
        '''删除一条数据'''
        rest = self.db.demo.test.delete_one({'name': 'ahn'})
        return rest

    def delete_many(self):
        '''删除多条数据'''
        rest = self.db.demo.test.delete_many({'sex': 'male'})
        return rest

    
def main():
    obj = DeleteMongo()
    #rest = obj.delete_one()
    rest = obj.delete_many()
    print(rest.deleted_count)
    
    
if __name__ == '__main__':
    main()

2


# 使用 mongoengine 操作 MongoDB

[MongoEngine 官方文档](http://docs.mongoengine.org/tutorial.html)

## ODM 简介

MongoEngine 是一个对象文档映射器（ODM），它大致相当于一个基于SQL的对象关系映射器（ORM）。MongoEngine提供的抽象是基于类的，所以你创建的所有模型都是类。

In [14]:
!pip install mongoengine

  Stored in directory: C:\Users\Diviner\AppData\Local\pip\Cache\wheels\0b\72\ba\e868acd0336c3800dae82e7a83d6e6c22cf20364a692514626
Successfully built mongoengine


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='pypi.org', port=443): Read timed out. (read timeout=15)")': /simple/mongoengine/
You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### 连接数据库

In [1]:
from mongoengine import connect

connect(
    db = 'demo',
    host = 'mongodb://localhost:27017'
)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

### 定义模型

In [2]:
from mongoengine import *

SEX_CHOICES = (
    ('male', '男'),
    ('female', '女')
)

class Grade(EmbeddedDocument):
    '''成绩'''
    subject = StringField(max_length=200, required=True)
    score = FloatField(required=True)

class Test(Document):
    name = StringField(max_length=200, required=True)
    age = IntField(required=True)
    sex = StringField(choices=SEX_CHOICES, required=True)
    grade = ListField(EmbeddedDocumentField(Grade))

## 增

In [31]:
from mongoengine import *

# 连接数据库
connect(
    db = 'demo',
    host = 'mongodb://localhost:27017'
)

# 定义模型
SEX_CHOICES = (
    ('male', '男'),
    ('female', '女')
)

class Grade(EmbeddedDocument):
    '''成绩'''
    subject = StringField(max_length=200, required=True)
    score = FloatField(required=True)

class Student(Document):
    name = StringField(max_length=200, required=True)
    age = IntField(required=True)
    sex = StringField(choices=SEX_CHOICES, required=True)
    grade = ListField(EmbeddedDocumentField(Grade))
    
    meta = {
        'collection': 'students'
    }
    
    
class TestMongoEngine(object):
    
    def add_one(self):
        '''新增一条记录'''
        math = Grade(subject='数学', score=95)
        stu_obj = Student(name="bob2", age=18, sex="female", grade=[math])
        stu_obj.save()
        return stu_obj
    

def main():
    obj = TestMongoEngine()
    rest = obj.add_one()
    print(rest.pk)
    
    
if __name__ == '__main__':
    main()

5e47671504ca2e36d435f7e3


## 查

In [37]:
from mongoengine import *

# 连接数据库
connect(
    db = 'demo',
    host = 'mongodb://localhost:27017'
)

# 定义模型
SEX_CHOICES = (
    ('male', '男'),
    ('female', '女')
)

class Grade(EmbeddedDocument):
    '''成绩'''
    subject = StringField(max_length=200, required=True)
    score = FloatField(required=True)

class Student(Document):
    name = StringField(max_length=200, required=True)
    age = IntField(required=True)
    sex = StringField(choices=SEX_CHOICES, required=True)
    grade = ListField(EmbeddedDocumentField(Grade))
    
    meta = {
        'collection': 'students',  # 存入数据库中的 students 文档
    }
    
    
class TestMongoEngine(object):
    
    def get_one(self):
        '''查询一条记录'''
        return Student.objects.first()
    
    def get_more(self):
        '''查询一条记录'''
        return Student.objects.filter(name='bob2').order_by('+age').all()
    
    def get_from_oid(self, oid):
        '''根据id查询数据'''
        return Student.objects.filter(pk=oid).first()
    

def main():
    obj = TestMongoEngine()
    
    #rest = obj.get_one()
    #print(rest.id)
    #print(rest.name)
    
    rest = obj.get_more()
    for item in rest:
        print(item.age)
    
    #rest = obj.get_from_oid('5e47605b04ca2e36d435f7e0')
    #print(rest.id)
    #print(rest.name)
    
    
if __name__ == '__main__':
    main()

16
16
16
18


## 改

In [40]:
from mongoengine import *

# 连接数据库
connect(
    db = 'demo',
    host = 'mongodb://localhost:27017'
)

# 定义模型
SEX_CHOICES = (
    ('male', '男'),
    ('female', '女')
)

class Grade(EmbeddedDocument):
    '''成绩'''
    subject = StringField(max_length=200, required=True)
    score = FloatField(required=True)

class Student(Document):
    name = StringField(max_length=200, required=True)
    age = IntField(required=True)
    sex = StringField(choices=SEX_CHOICES, required=True)
    grade = ListField(EmbeddedDocumentField(Grade))
    
    meta = {
        'collection': 'students'
    }
    
    
class TestMongoEngine(object):
    
    def update(self):
        '''修改数据'''
        # 修改一条数据
        # rest = Student.objects.filter(name='bob2').update_one(inc__age=1)
        # return rest
        # 修改多条数据
        rest = Student.objects.filter(name='bob2').update(inc__age=1)
        return rest 
    

def main():
    obj = TestMongoEngine()
    rest = obj.update()
    print(rest)  # 影响的行数
    
    
if __name__ == '__main__':
    main()

4


## 删

In [41]:
from mongoengine import *

# 连接数据库
connect(
    db = 'demo',
    host = 'mongodb://localhost:27017'
)

# 定义模型
SEX_CHOICES = (
    ('male', '男'),
    ('female', '女')
)

class Grade(EmbeddedDocument):
    '''成绩'''
    subject = StringField(max_length=200, required=True)
    score = FloatField(required=True)

class Student(Document):
    name = StringField(max_length=200, required=True)
    age = IntField(required=True)
    sex = StringField(choices=SEX_CHOICES, required=True)
    grade = ListField(EmbeddedDocumentField(Grade))
    
    meta = {
        'collection': 'students'
    }
    
    
class TestMongoEngine(object):
    
    def delete(self):
        '''删除数据'''
        # 删除一条数据
        # return Student.objects.filter(name='bob2').first().delete()
        # 删除多条数据
        return Student.objects.filter(name='bob2').delete()
    

def main():
    obj = TestMongoEngine()
    rest = obj.delete()
    print(rest)
    
    
if __name__ == '__main__':
    main()

4


# 数据实例

In [ ]:
db.demo.insertMany(
    [
        { name:"bob", age:16, sex:"male", grade:95},
        { name:"ahn", age:18, sex:"female", grade:45},
        { name:"xi", age:15, sex:"male", grade:75},
        { name:"bob1", age:16, sex:"male", grade:95},
        { name:"ahn1", age:18, sex:"male", grade:45},
        { name:"xi1", age:15, sex:"female", grade:55},
        { name:"bob2", age:16, sex:"female", grade:95},
        { name:"ahn2", age:18, sex:"male", grade:60},
        { name:"xi2", age:15, sex:"male", grade:75},
        { name:"bob3", age:16, sex:"male", grade:95},
        { name:"ahn3", age:18, sex:"female", grade:45},
        { name:"xi3", age:15, sex:"male", grade:85},
        { name:"bob4", age:16, sex:"female", grade:95},
        { name:"ahn4", age:18, sex:"male", grade:45},
        { name:"xi4", age:15, sex:"male", grade:75}
    ]
)